In [1]:
from collections import Counter

In [2]:
from sklearn_hierarchical_classification.constants import ROOT

In [3]:
import pickle as pk
def save_obj(obj, name):
    """
    Saving the pickle object
    """
    with open(name + '.pkl', 'wb') as file:
        pk.dump(obj, file, pk.HIGHEST_PROTOCOL)


def load_obj(name):
    """
    loading the pickle object
    """
    with open(name + '.pkl', 'rb') as file:
        return pk.load(file)

In [4]:
yamtools = load_obj('/home/nnabizad/code/toolpred/data/yam/yam_tools')
ings = load_obj('/home/nnabizad/code/toolpred/data/yam/yam_ings')

In [99]:
import string as st
import re
def cleaing(txt):
    words = txt.split()
    for word in words:
        if word in st.punctuation: words.remove(word)
        if word == 'and': words.remove(word)
    txt = ' '.join(words)
    re.sub(r'([^\s\w-]|_)+', '', txt)
    return txt

In [122]:
allrec = []
for rec in ings:
    _rec = []
    for step in rec:
        _step =[]
        for _ing in step:
            _step.append(cleaing(_ing))
        _rec.append(set(_step))
    allrec.append(_rec)

In [5]:
from collections import defaultdict

In [367]:
def hir_create(counts):
    class_hierarchy = defaultdict(list)
    for part in counts:
        if counts[part]>1:
            words = part.split()
            if len(words)>1:
                if ' '.join(words[0:-1]) not in class_hierarchy[words[-1]]:
                    class_hierarchy[words[-1]].append(' '.join(words[0:-1]))
            else:
                class_hierarchy["<ROOT>"].append(words[0])
    return class_hierarchy

In [602]:
def remove_ones(class_hierarchy, root):
    for tool in class_hierarchy.copy():
        if tool != root:
            if len(class_hierarchy[tool])<2 and class_hierarchy[tool][0]+ ' ' + tool not in class_hierarchy[root]:
                class_hierarchy[root].append(class_hierarchy[tool][0]+ ' ' + tool)
                del class_hierarchy[tool]
            else:
                if tool in class_hierarchy[root]:
                    class_hierarchy[tool].append('None')
                else:
                    class_hierarchy[root].append(tool)
    return class_hierarchy

In [501]:
def next_level(tclass_hierarchy, root):
    keys = list(tclass_hierarchy.keys())
    keys.remove(root)
    for tool in keys:
        for subtool in tclass_hierarchy[tool][:]:
            words = subtool.split()
            if len(words)>1:
                if words[-1]+' '+tool not in tclass_hierarchy[tool]:
                    tclass_hierarchy[tool].append(words[-1]+' '+tool)
                tclass_hierarchy[words[-1]+' '+tool].append(' '.join(words[0:-1]))
                tclass_hierarchy[tool].remove(subtool)
    return tclass_hierarchy

In [600]:
def clean_levels(tclass_hierarchy, root):
    keys = list(tclass_hierarchy.keys())
    keys.remove(root)
    for tool in list(tclass_hierarchy.keys()):
        if len(tclass_hierarchy[tool])<2:
            if tclass_hierarchy[tool][0] in tclass_hierarchy.keys():
                if tool in tclass_hierarchy[root]: tclass_hierarchy[root].remove(tool)
                tclass_hierarchy[root].append(tclass_hierarchy[tool][0])
                del tclass_hierarchy[tool]
            else:
                for key in keys:
                    if tool in tclass_hierarchy[key] and tclass_hierarchy[tool]:
                        tclass_hierarchy[key].remove(tool)
                        tclass_hierarchy[key].append((' '.join(tclass_hierarchy[tool][0].split()[:-1]) + ' ' + tool).strip())
                        del tclass_hierarchy[tool]
    return tclass_hierarchy

In [503]:
def isleaf(node,hierarchy):
    if node in hierarchy.keys():
        return False
    else:
        return True

In [504]:
def check_dublicates(class_hierarchy):
    myset = set()
    for i in class_hierarchy:
        for j in class_hierarchy[i]:
            if j in myset and j not in class_hierarchy.keys():
                print(j)
            elif j not in class_hierarchy.keys():
                myset.add(j)

In [629]:
def addparents(i,tool):
    if i.split()[-1] != tool.split()[0]:
        return i + ' ' + tool
    elif i.split()[-1] != tool.split()[-1]:
        return i + ' ' + tool.split()[-1]
    else:
        return i

In [588]:
def save_dic(dic, name):
    mydic = dict()
    keys = list(dic.keys())
    keys.remove('<ROOT>')
    mydic['<ROOT>'] = dic['<ROOT>']
    for tool in keys:
        if len(dic[tool])>0:
            mydic[tool] = list(set([addparents(i,tool) for i in dic[tool]]))
    save_obj(mydic, name)
    return mydic

In [609]:
mactools = load_obj('/home/nnabizad/code/toolpred/data/mac/mac_tools')
allmactools = [i for j in mactools for k in j for i in k]
toolscounts = Counter(allmactools)
toolset = set(allmactools)

In [630]:
toolclass_hierarchy = hir_create(toolscounts)

In [631]:
toolclass_hierarchy = remove_ones(toolclass_hierarchy, root="<ROOT>")

In [632]:
toolclass_hierarchy = next_level(toolclass_hierarchy, root="<ROOT>")

In [633]:
toolclass_hierarchy = clean_levels(toolclass_hierarchy, root="<ROOT>")

In [626]:
toolclass_hierarchy

defaultdict(list,
            {'<ROOT>': ['spudger',
              'coin',
              'paperclip',
              'pin',
              'tweezers',
              'screwdriver',
              'iopener',
              'probe_and_pickset',
              'arctic silver thermal paste',
              'opening tool',
              'plastic cards',
              'arctic silver arcticlean',
              'heat gun',
              'heavy-duty suction cups pair',
              'knife',
              'wire striper',
              'utility scissors',
              'ipad battery isolation pick',
              'hair dryer',
              'suction handle',
              'microfiber cleaning cloth',
              'usb cable',
              '99 isopropyl alcohol',
              'adhesive remover',
              'isopropyl alcohol wipes',
              'nut driver'],
             'screwdriver': ['None',
              'phillips screwdriver',
              'torx screwdriver',
              'tri-point scre

In [615]:
check_dublicates(toolclass_hierarchy)

In [635]:
lst = save_dic(toolclass_hierarchy, 'mactool_hi')
lst

{'<ROOT>': ['spudger',
  'coin',
  'paperclip',
  'pin',
  'tweezers',
  'screwdriver',
  'iopener',
  'probe_and_pickset',
  'arctic silver thermal paste',
  'opening tool',
  'plastic cards',
  'arctic silver arcticlean',
  'heat gun',
  'heavy-duty suction cups pair',
  'knife',
  'wire striper',
  'utility scissors',
  'ipad battery isolation pick',
  'hair dryer',
  'suction handle',
  'microfiber cleaning cloth',
  'usb cable',
  '99 isopropyl alcohol',
  'adhesive remover',
  'isopropyl alcohol wipes',
  'nut driver'],
 'screwdriver': ['None screwdriver',
  'pentalobe screwdriver',
  'torx screwdriver',
  '1.5mm hex screwdriver',
  'tri-point screwdriver',
  '2.5mm flathead screwdriver',
  'phillips screwdriver'],
 'knife': ['tech knife', 'precision utility knife'],
 'phillips screwdriver': ['ph00 phillips screwdriver',
  'ph000 phillips screwdriver',
  'ph0 phillips screwdriver'],
 'torx screwdriver': ['t3 torx screwdriver',
  't6 torx screwdriver',
  't4 torx screwdriver',
  '

In [636]:
for key in lst:
    print(key, lst[key])
    print()

<ROOT> ['spudger', 'coin', 'paperclip', 'pin', 'tweezers', 'screwdriver', 'iopener', 'probe_and_pickset', 'arctic silver thermal paste', 'opening tool', 'plastic cards', 'arctic silver arcticlean', 'heat gun', 'heavy-duty suction cups pair', 'knife', 'wire striper', 'utility scissors', 'ipad battery isolation pick', 'hair dryer', 'suction handle', 'microfiber cleaning cloth', 'usb cable', '99 isopropyl alcohol', 'adhesive remover', 'isopropyl alcohol wipes', 'nut driver']

screwdriver ['None screwdriver', 'pentalobe screwdriver', 'torx screwdriver', '1.5mm hex screwdriver', 'tri-point screwdriver', '2.5mm flathead screwdriver', 'phillips screwdriver']

knife ['tech knife', 'precision utility knife']

phillips screwdriver ['ph00 phillips screwdriver', 'ph000 phillips screwdriver', 'ph0 phillips screwdriver']

torx screwdriver ['t3 torx screwdriver', 't6 torx screwdriver', 't4 torx screwdriver', 't8 torx screwdriver', 't10 torx screwdriver', 't5 torx screwdriver', 't9 torx screwdriver']


In [637]:
macparts = load_obj('/home/nnabizad/code/toolpred/data/mac/mac_parts')
allmacparts = [i for j in macparts for k in j for i in k]
partsscounts = Counter(allmacparts)

In [638]:
partclass_hierarchy = hir_create(partsscounts)

In [639]:
partclass_hierarchy = remove_ones(partclass_hierarchy, root="<ROOT>")

In [640]:
partclass_hierarchy = next_level(partclass_hierarchy, root="<ROOT>")

In [641]:
partclass_hierarchy = clean_levels(partclass_hierarchy, root="<ROOT>")

In [645]:
lst = save_dic(partclass_hierarchy, 'macpart_hi')
for key in lst:
    print(key, lst[key])
    print()

<ROOT> ['keyboard', 'harddrive', 'processor', 'modem', 'fan', 'battery', 'pmu', 'post', 'shield', 'case', 'opticaldrive', 'screw', 'drive', 'bracket', 'bezel', 'standoff', 'slot', 'magnet', 'chip', 'board', 'display', 'antenna', 'cable', 'tape', 'nut', 'ribbon', 'casing', 'subwoofer', 'inverter', 'motherboard', 'speaker', 'connector', 'microphone', 'lcd', 'assembly', 'trackpad', 'ssd', 'isight', 'panel', 'heatsink', 'cover', 'gasket', 'interposer', 'plate', 'sticker', 'hatch', 'expansion bay module', 'upper casing', 'card', 'small black plastic rectangle', 'pram battery', 'ram chip', 'silver', 'framework', 'cage', 'cd-rom drive tray', 'upper case', 'trackpad support', 'black plastic bar', 'metal heat exchanger', 'fan or placeholder', 'foot', 'metal ring', 'cap', 'small greasy spring', 'grill', 'heat sink', 'door', 'function key', 'sleep light', 'bumper', 'spacer', 'left io frame', 'mount', 'display inverter', 'translucent plastic sheath', 'sensor', 'pad', 'metal c-channel', 'mid wall',

In [644]:
check_dublicates(lst)